## Goal: get SNPs both CLOSE to SJs (exonic, +- 10 bp) and within SSs themselves; only for annotated SJs

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [6]:
config = load_config()
od = '../../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [ ]:
gtf_file = proc_cfg(config['ref']['gtf'], od)


In [11]:
# read in ref. gtf and get SJs from there
df = pr.read_gtf(gtf_file)

# get the introns
df = df.features.introns()
df = df.df
df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,exon_number,exon_id,transcript_support_level,havana_transcript,hgnc_id,havana_gene,ont,protein_id,ccdsid,artif_dupl
0,GL000008.2,HAVANA,gene,83331,200303,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GL000008.2,HAVANA,transcript,83331,200303,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GL000008.2,HAVANA,exon,83331,83545,.,+,.,ENSG00000296732.1,lncRNA,...,1,ENSE00004078666.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GL000008.2,HAVANA,exon,85566,85625,.,+,.,ENSG00000296732.1,lncRNA,...,2,ENSE00004078618.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GL000008.2,HAVANA,exon,199427,200303,.,+,.,ENSG00000296732.1,lncRNA,...,3,ENSE00004078645.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GL000008.2,HAVANA,transcript,83332,130626,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GL000008.2,HAVANA,exon,83332,83545,.,+,.,ENSG00000296732.1,lncRNA,...,1,ENSE00004078614.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GL000008.2,HAVANA,exon,85566,85625,.,+,.,ENSG00000296732.1,lncRNA,...,2,ENSE00004078618.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


AttributeError: 'function' object has no attribute 'df'